In [1]:
import os
import re
import json
from itertools import groupby, combinations, chain, permutations
import numpy as np

In [7]:
aggregate_fn = lambda rewards: rewards[0] * np.sum(rewards[1:])

In [75]:
data = []
with open('/Users/jayceping/code/T2I_GRPO/Flow_GRPO/dataset/T2IS/data_2by2.jsonl', 'r') as f:
    for line in f:
        data.append(json.loads(line))
print(len(data))

285


In [148]:
similar_entity_set = [
    # 人物相关
    ('boys', 'girls', 'women', 'men', 'ladies', 'gentlemen'),
    ('boy', 'girl', 'woman', 'man', 'lady', 'gentleman'),
    ('child', 'kid', 'toddler', 'teenager'),
    ('teacher', 'student', 'worker', 'boss'),
    ('king', 'queen', 'prince', 'princess'),
    ('doctor', 'nurse'),
    ('artist', 'painter', 'sculptor', 'musician'),
    ('hero', 'villain', 'warrior', 'knight'),

    # 动物
    ('cat', 'kitten', 'dog', 'puppy'),
    ('lion', 'tiger', 'leopard', 'cheetah'),
    ('wolf', 'fox', 'coyote', 'jackal'),
    ('horse', 'pony', 'donkey', 'zebra'),
    ('bird', 'eagle', 'hawk', 'owl'),
    ('fish', 'shark', 'dolphin', 'whale'),
    ('cow', 'bull', 'sheep', 'goat'),
    ('chicken', 'duck', 'goose', 'turkey'),

    # 交通工具
    ('car', 'truck', 'motorcycle', 'bicycle'),
    ('bus', 'train', 'tram', 'subway'),
    ('ship', 'boat', 'yacht', 'canoe'),
    ('airplane', 'jet', 'helicopter', 'glider'),

    # 食物
    ('apple', 'banana', 'orange', 'grape'),
    ('bread', 'cake', 'cookie', 'pastry'),
    ('pizza', 'burger', 'sandwich', 'hotdog'),
    ('coffee', 'tea', 'milk', 'juice'),
    ('rice', 'noodles', 'pasta', 'dumplings'),

    # 家具 & 室内
    ('table', 'chair', 'sofa', 'couch'),
    ('bed', 'desk', 'wardrobe', 'cabinet'),
    ('lamp', 'candle', 'lantern', 'chandelier'),
    ('window', 'door', 'curtain'),

    # 自然景物
    ('mountain', 'hill', 'valley', 'cliff'),
    ('river', 'lake', 'pond', 'ocean'),
    ('forest', 'jungle', 'woods', 'grove'),
    ('sun', 'moon', 'star', 'planet'),

    # 抽象特征/形容词
    ('old', 'young'),
    ('happy', 'sad', 'angry', 'calm'),
    ('big', 'small', 'tiny', 'huge'),
    ('fast', 'slow', 'quick', 'lazy'),
    ('beautiful', 'ugly', 'pretty', 'handsome'),
    ('hot', 'cold', 'warm', 'cool'),
]

def gen_new_prompt(prompts):
    if isinstance(prompts, str):
        prompts = [prompts]

    new_prompts = set([p for p in prompts])
    for entities in similar_entity_set:
        for e1, e2 in permutations(entities, 2):
            # Replace word with boundary
            new_prompts.update(re.sub(r'\b' + re.escape(e1) + r'\b', e2, prompt) for prompt in prompts)

    return new_prompts

def fix_point(fn, x0, max_iter = 10):

    x1 = fn(x0)
    while x1 != x0 and max_iter > 0:
        x0, x1 = x1, fn(x1)
        max_iter -= 1

    return x1

In [149]:
prompt_data = [(d['idx'], d['prompt']) for d in data]

In [150]:
new_prompts = []
for idx, prompt in prompt_data:
    new_prompts.append(fix_point(gen_new_prompt, prompt, max_iter=1))

In [151]:
# all_new_prompts = list(set(chain(*new_prompts)))
all_new_prompts = list(chain(*new_prompts))

In [152]:
len(all_new_prompts)

6850